In [28]:
import requests
import json
import pandas as pd
import os
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

In [29]:
# This code line allows to remove warning produce by verify = False
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [30]:
# Define the export function
def generatecsvFiles(export_df,variable):
    for estaciones_id in export_df['id_estacion'].unique():
        data = export_df.query("id_estacion == @estaciones_id")
        data[['id_estacion','fecha','indice_calidad','indice_revision','tipo_calculo','valor']].to_csv(f'output/{estaciones_id}_{variable}.csv',float_format='%.3f',index=False)

In [31]:
archivo = 'run_info.xml'
if os.path.exists(archivo):
    with open(archivo) as f:
        root = ET.parse(f)
        for child in root.iter('*'):
            if child.tag == '{http://www.wldelft.nl/fews/PI}startDateTime':
                fecha = child.attrib['date'] + ' ' + child.attrib['time']
                break
        fin = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        inicio = fin - timedelta(days = 3)
else:
    # Aquí se pueden ingresar un período de fechas para la consulta de niveles
    # Si se deja vacío, la consulta será a las últimas 24 horas
    ###################################
    # PERÍODO (máximo 15 días) #
    # Formato yyyy-mm-dd hh:mm #
    ###################################
    #inicio = '2021-01-01 00:00' #
    #fin = '2021-01-14 00:00' #
    inicio = ''
    fin = ''
    ###################################
    # Si inicio o fin se dejaron vacías, se haya la fecha actual (fin) y un día antes (inicio)
    if inicio == '' or fin == '':
        fin = datetime.today().replace(second=0, microsecond=0)
        inicio = fin - timedelta(days=3)
print(inicio, fin)

2025-07-19 16:58:00 2025-07-22 16:58:00


In [32]:
# read api url and credentials from .env
from dotenv import load_dotenv

load_dotenv()

True

In [33]:
# Dinagua Web Service
url_base = os.environ["url"] # 'https://www.ambiente.gub.uy/dinaguaws/'

In [34]:
# Username and password
user_dinagua = {"user": os.environ["user"], "password": os.environ["password"]}

In [35]:
# Get the token from DINAGUA WS
response_token = requests.post(url=url_base + "gettoken", json=user_dinagua,verify=False);
response_token.close()

In [36]:
# Extract token and assign it to a variable
token = response_token.text

In [37]:
# Add the token to a bearer authorization
headers = {"Authorization" : "Bearer " + token}

get dinagua stations

In [38]:
# Get Dinagua stations
response_estaciones = requests.get(url=url_base + "service/estaciones",json=user_dinagua, headers = headers, verify=False)
response_estaciones.close()

In [39]:
# Convert it to json and dataframe variables
estaciones_dinagua = json.loads(response_estaciones.content.decode('utf-8'))
estaciones_dinagua = pd.DataFrame(estaciones_dinagua)

In [40]:
# export csv
estaciones_dinagua.to_csv("output/estaciones.csv", index=False)

In [41]:
# Filter station by Activa
estaciones_activa = estaciones_dinagua.query("estado == 'Activa'")
# Get id stations
id_estaciones_activa = estaciones_activa["id_estacion"]
# Get name of the station
nombre_estaciones_activa = estaciones_activa["apodo"]

In [42]:
display(estaciones_dinagua.query("id_estacion == 51"))

,apodo,area_cuenca_km2,cuenca_nivel_5,nombre_curso_observado,id_estacion,departamento,latitud,longitud,estado,ultima_fecha,fecha_inicio,entidad,pais,tipo,var_corto,variables,zona_horaria
171,Paso de los Toros,40123.0,55135,Río Negro,51,TACUAREMBÓ,-32.8204,-56.51116,Activa,2025-04-30,1960-06-01,Servicio hidrológico,Uruguay,Limnimétrica,H,Nivel,GMT-3


In [43]:
id_estaciones_activa.to_clipboard()

In [44]:
# Define the parameters for the get niveles end-point
headers_niveles = {"inicio" : inicio.strftime('%Y-%m-%d %H:%M'), "fin" : fin.strftime('%Y-%m-%d %H:%M'), "variable" : "H"}
headers_caudales = {"inicio" : inicio.strftime('%Y-%m-%d %H:%M'), "fin" : fin.strftime('%Y-%m-%d %H:%M'), "variable" : "Q"}
headers_temperatura = {"inicio" : inicio.strftime('%Y-%m-%d %H:%M'), "fin": fin.strftime('%Y-%m-%d %H:%M'), "variable" : "T"}
headers_diarios = {"inicio" : inicio.strftime('%Y-%m-%d %H:%M'), "fin" : fin.strftime('%Y-%m-%d %H:%M'), "variable" : "Q","tipo_calculo":"Promedio"}

In [45]:
# Get niveles from Dinagua WS
response_niveles = requests.get(url= url_base + "service/datos/horarios", params=headers_niveles, json=user_dinagua, headers = headers, verify=False)
# Close the connection
response_niveles.close()
# Convert it to json and dataframe
response_niveles = json.loads(response_niveles.content.decode('utf-8'))
response_niveles = pd.DataFrame(response_niveles)

In [46]:
response_niveles.query("id_estacion == 1330")

,id_estacion,apodo,fecha,indice_calidad,indice_revision,tipo_calculo,valor,variable,unidades,zona_horaria,variablecorto
1,1330,Santa Lucia R-11,19-07-2025 17:00:00,Dato aceptado,Datos brutos,Ingresado,1.57,Nivel,m,GMT-3,H
24,1330,Santa Lucia R-11,19-07-2025 17:30:00,Dato aceptado,Datos brutos,Ingresado,1.57,Nivel,m,GMT-3,H
55,1330,Santa Lucia R-11,19-07-2025 18:00:00,Dato aceptado,Datos brutos,Ingresado,1.58,Nivel,m,GMT-3,H
87,1330,Santa Lucia R-11,19-07-2025 18:30:00,Dato aceptado,Datos brutos,Ingresado,1.58,Nivel,m,GMT-3,H
100,1330,Santa Lucia R-11,19-07-2025 19:00:00,Dato aceptado,Datos brutos,Ingresado,1.58,Nivel,m,GMT-3,H
...,...,...,...,...,...,...,...,...,...,...,...
2538,1330,Santa Lucia R-11,22-07-2025 04:00:00,Dato aceptado,Datos brutos,Ingresado,1.58,Nivel,m,GMT-3,H
2549,1330,Santa Lucia R-11,22-07-2025 04:30:00,Dato aceptado,Datos brutos,Ingresado,1.58,Nivel,m,GMT-3,H
2556,1330,Santa Lucia R-11,22-07-2025 05:00:00,Dato aceptado,Datos brutos,Ingresado,1.58,Nivel,m,GMT-3,H
2563,1330,Santa Lucia R-11,22-07-2025 05:30:00,Dato aceptado,Datos brutos,Ingresado,1.57,Nivel,m,GMT-3,H


In [47]:
# Get Caudales from Dinagua WS
response_caudales = requests.get(url= url_base + "service/datos/horarios", params=headers_caudales, json=user_dinagua, headers = headers, verify=False)
# Close the connection
response_caudales.close()
# Convert it to json and dataframe
response_caudales = json.loads(response_caudales.content.decode('utf-8'))
response_caudales = pd.DataFrame(response_caudales)

In [48]:
# Get Temperatura from Dinagua WS
response_temperature = requests.get(url= url_base + "service/datos/horarios", params=headers_temperatura, json=user_dinagua, headers = headers, verify=False)
# Close the connection
response_temperature.close()
# Convert it to json and dataframe
response_temperature = json.loads(response_temperature.content.decode('utf-8'))
response_temperature = pd.DataFrame(response_temperature)

In [49]:
display(response_temperature)

,id_estacion,apodo,fecha,indice_calidad,indice_revision,tipo_calculo,valor,variable,unidades,zona_horaria,variablecorto
0,1330,Santa Lucia R-11,19-07-2025 18:00:00,Dato aceptado,Datos brutos,Ingresado,11.8,Temperatura en agua,°C,GMT-3,T
1,190,Carmelo - Atracadero,19-07-2025 18:00:00,Dato aceptado,Datos brutos,Ingresado,11.7,Temperatura en agua,°C,GMT-3,T
2,960,Paso Borches,19-07-2025 18:00:00,Dato aceptado,Datos brutos,Ingresado,13.0,Temperatura en agua,°C,GMT-3,T
3,671,Laguna Garzón,19-07-2025 18:00:00,Dato aceptado,Datos brutos,Ingresado,12.6,Temperatura en agua,°C,GMT-3,T
4,450,Lag. del Sauce (vertedero),19-07-2025 18:00:00,Dato aceptado,Datos brutos,Ingresado,12.7,Temperatura en agua,°C,GMT-3,T
...,...,...,...,...,...,...,...,...,...,...,...
226,660,Paso de las Toscas,22-07-2025 06:00:00,Dato aceptado,Datos brutos,Ingresado,12.5,Temperatura en agua,°C,GMT-3,T
227,671,Laguna Garzón,22-07-2025 06:00:00,Dato aceptado,Datos brutos,Ingresado,12.7,Temperatura en agua,°C,GMT-3,T
228,440,Fray Marcos,22-07-2025 12:00:00,Dato aceptado,Datos brutos,Ingresado,11.8,Temperatura en agua,°C,GMT-3,T
229,671,Laguna Garzón,22-07-2025 12:00:00,Dato aceptado,Datos brutos,Ingresado,12.8,Temperatura en agua,°C,GMT-3,T


In [50]:
# Get Daily Discharge
response_daily = requests.get(url= url_base + "service/datos/diarios", params=headers_diarios, json=user_dinagua, headers = headers, verify=False)
# Close the connection
response_daily.close()
# Convert it to json and dataframe
response_daily = json.loads(response_daily.content.decode('utf-8'))
response_daily = pd.DataFrame(response_daily)

In [51]:
response_daily.query("id_estacion == 1330")

,id_estacion,apodo,fecha,indice_calidad,indice_revision,tipo_calculo,valor,variable,unidades,zona_horaria,variablecorto
10,1330,Santa Lucia R-11,20-07-2025 00:00:00,Dato aceptado,Datos brutos,Promedio,19.883953,Caudal,m3/s,GMT-3,Q
14,1330,Santa Lucia R-11,21-07-2025 00:00:00,Dato aceptado,Datos brutos,Promedio,19.191223,Caudal,m3/s,GMT-3,Q
25,1330,Santa Lucia R-11,22-07-2025 00:00:00,Dato aceptado,Datos brutos,Promedio,16.652160,Caudal,m3/s,GMT-3,Q


In [52]:
response_niveles[['id_estacion','fecha','indice_calidad','indice_revision','tipo_calculo','valor']].to_csv(f'output/niveles.csv',float_format='%.3f',index=False)
response_caudales[['id_estacion','fecha','indice_calidad','indice_revision','tipo_calculo','valor']].to_csv(f'output/caudales.csv',float_format='%.3f',index=False)
response_temperature[['id_estacion','fecha','indice_calidad','indice_revision','tipo_calculo','valor']].to_csv(f'output/temperatura.csv',float_format='%.3f',index=False)